In [7]:
import datetime
import time
from datetime import date

import pickle

import pandas as pd
import numpy as np

import os
import sys
import shutil

import matplotlib.pyplot as plt

In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서

from bs4 import BeautifulSoup as bs
import requests

In [27]:
def collect_info(soup):
    condition = {}
    condition['회사명'] = soup.head.title.text[:-6]
    h2_sel = '#__next main div.title-form__group > h2'
    for h2 in soup.select(h2_sel):
#         print(h2.text)
        if h2.text == '모집조건':
            dl_dd = h2.parent.parent.next_sibling.select('dl')
            for dd in dl_dd:
#                 print("***", dd.dt.text, "---", dd.dd.text)
                condition[dd.dt.text] = dd.dd.text

# 모집분야가 표시가 되어 있지  않으면 다음 칸의 업직종 내용으로 대체함.
        try :
            condition['모집분야']
        except:
            if h2.text == '근무조건':
                dl_dd = h2.parent.parent.next_sibling.select('dl')
                for dd in dl_dd:
                    if dd.dt.text == '업직종':
                        condition['모집분야'] = dd.dd.text
                    
        if h2.text == '근무지역':
            dl_dd = h2.parent.parent.parent.select('p')
#             print("ppp", dl_dd)
            for dd in dl_dd:
#                 print("***", '근무지역', "---", dd.text)
                condition['근무지역'] = dd.text
            
    return condition

In [3]:
service = Service(r"116/chromedriver.exe")
options = wd.ChromeOptions()
driver = wd.Chrome(service=service, options=options)

In [4]:
# url = 'https://www.albamon.com/'
# driver.get(url)
# # driver.maximize_window()

In [5]:
# login 화면
url = 'https://www.albamon.com/user-account/login?linkType=logout&memberType=PERSONAL'
driver.get(url)

In [8]:
myID = os.environ['albamon_id']
id_sel = '#memberId'
id_element = driver.find_element(By.CSS_SELECTOR, id_sel)
id_element.clear()
id_element.send_keys(myID)
id_element.send_keys(Keys.ENTER) # 이 것을 실행시키면 안내 화면이 뜨고 '화면' 버튼을 클릭하여야 함. 따라서 제외시킴

In [9]:
alert = driver.switch_to.alert
alert.accept()
time.sleep(1)

In [10]:
myPass = os.environ['albamon_pass']
pass_sel = '#memberPassword'
pass_element = driver.find_element(By.CSS_SELECTOR, pass_sel)
pass_element.clear()
pass_element.send_keys(myPass)
pass_element.send_keys(Keys.ENTER)

In [ ]:
# log_button_sel = '#__next  main div.Login_login-btn-wrap__la071 > button'
# log_button_element = driver.find_element(By.CSS_SELECTOR, log_button_sel)
# log_button_element.click()

In [12]:
q_sel = '#__next header form > div > div > div > input[type=text]'
q_element = driver.find_element(By.CSS_SELECTOR, q_sel) # 검색 창 element 선택

In [14]:
q_element.clear()
# q_element.send_keys('시니어') # 검색어 입력
search_item = input("검색할 알바를 입력하세요 예: '시니어 대전' : ")
if search_item == '':
    search_item = '시니어 대전'
q_element.send_keys(search_item) # 검색어 입력
q_element.send_keys(Keys.ENTER)

검색할 알바를 입력하세요 예: '시니어 대전' : 


In [15]:
# 검색 결과 갯수
num_s = driver.find_element(By.CSS_SELECTOR,'#__next  main div.list-header__item > span > strong').text.strip()
num_search = int(num_s)

# 페이지 갯수
num_page = divmod(num_search, 50)[0] + 1  # 몫에 1을 더함.

In [16]:
driver.find_element(By.ID, 'select-size').click() # 한 페이지에 최대 갯수 선택
driver.find_element(By.ID, 'option-button--3').click() # 최대 50개 선택

In [17]:
# button = "#__next  main button[type=button][aria-current]"  # 하단의 페이지 갯수 번호 최대 5개씩 표현됨.
# b_element = driver.find_elements(By.CSS_SELECTOR, button)

In [18]:
url_ = []
button = "#__next  main button[type=button][aria-current]" # 하단의 페이지 갯수 번호 최대 5개씩 표현됨.
b_element = driver.find_elements(By.CSS_SELECTOR, button)  # 페이지가 변할 때마다 지정을 해 줘야 함. 
for i in range(0, num_page):
    time.sleep(2)
    button = "#__next  main button[type=button][aria-current]" # 하단의 페이지 갯수 번호 최대 5개씩 표현됨.
    b_element = driver.find_elements(By.CSS_SELECTOR, button)  # 페이지가 변할 때마다 지정을 해 줘야 함. 
    b_element[i].click()
#     driver.refresh()
    time.sleep(2)
    
    url_sel = '#__next main ul.SimpleRecruitList_simple-recruit-list__ul__5KKhf li a[target=_self]'
    url_element = driver.find_elements(By.CSS_SELECTOR, url_sel) # https: 기 있는 모든 tag a target='_self' 선택
    
    # 한 페이지에 있는 최대 갯수 50개 url 선택
    for u in url_element:
        url_.append(u.get_attribute('href'))

In [19]:
len(url_)

19

In [28]:
column_sel = ['회사명', '모집마감', '모집인원', '모집분야', '근무지역', '성별', '학력', '연령', '우대사항']
df_con = pd.DataFrame(columns = column_sel)

for url in url_:
    r = requests.get(url)
    html = r.content
    soup = bs(html, 'html.parser')
    df_temp = pd.DataFrame([collect_info(soup)])
    df_con = pd.concat([df_con, df_temp])
    
df_con.set_index(np.arange(len(df_con.index)), inplace=True)

In [31]:
df_con[:7]

,회사명,모집마감,모집인원,모집분야,근무지역,성별,학력,연령,우대사항
0,"[㈜우림맨테크] | [명품관]신세계 대전점/판매사원/주니어,시니어/ 신입,경력자/남성매장",상시모집,00명(인원미정),"매장관리·판매, 패션·잡화디자인, 백화점·쇼핑몰",대전 유성구 엑스포로 1 (도룡동) 신세계백화점 대전점 D&G,성별무관,학력무관,연령무관,"유사업무 경험 우대, 인근 거주 우대"
1,"[㈜우림맨테크] | [명품관]대전 현대 아울렛점/주니어,시니어/신입가능/주5일근무",상시모집,00명(인원미정),"매장관리·판매, 패션·잡화디자인, 백화점·쇼핑몰","대전 유성구 테크노중앙로 123 (용산동, 현대프리미엄아울렛대전점) 명품관 마이클코어스",성별무관,학력무관,연령무관,"유사업무 경험 우대, 인근 거주 우대"
2,[PXG] | 갤러리아타임월드에서 시니어를 모집합니다,상시모집,1명,매장관리·판매,"대전 서구 대덕대로 211 (둔산동, 갤러리아타임월드백화점) 7층PXG",성별무관,학력무관,28세 ~ 45세,NaN
3,[신세계 대전점 디스커버리 키즈] | [신세계백화점 디스커버리 키즈] 매장관리 및 ...,2023-10-30마감 16일전,0명(인원미정),"백화점·쇼핑몰, 매장관리·판매, 의류·잡화·쥬얼리매장",대전 유성구 엑스포로 1 (도룡동)대전 유성구 엑스포로 1,성별무관,학력무관,연령무관,유사업무 경험 우대
4,[호야 & 피플] | [SW19] 갤러리아 타임월드점 시니어 채용합니다.,상시모집,1명,매장관리·판매,전국,성별무관,학력무관,연령무관,NaN
5,[㈜룸페커] | (주)룸페커 대전노은 직영점 매장 시니어 정직원 구인,2023-10-31마감 17일전,1명,"매장판매, 매장관리",대전 유성구 은구비남로33번길 51 (지족동) 1층,성별무관,고등학교 졸업 이상,연령무관,"유사업무 경험 우대, 인근 거주 우대"
6,[㈜ 에프에이모스트] | 스말트 롯데백화점 대전점 시니어 / 파트타이머 구인 (남...,상시모집,3명,"매니저, 시니어, 주3일 파트타이머","대전 서구 계룡로 598 (괴정동, 롯데백화점) 롯데백화점 대전점 1층 SMALT",성별무관,학력무관,연령무관,"영어 가능, 엑셀 가능, 유사업무 경험 우대"


In [21]:
df_con[:]

,회사명,모집마감,모집인원,모집분야,근무지역,성별,학력,연령,우대사항
0,"[㈜우림맨테크] | [명품관]신세계 대전점/판매사원/주니어,시니어/ 신입,경력자/남성매장",상시모집,00명(인원미정),NaN,대전 유성구 엑스포로 1 (도룡동) 신세계백화점 대전점 D&G,성별무관,학력무관,연령무관,"유사업무 경험 우대, 인근 거주 우대"
1,"[㈜우림맨테크] | [명품관]대전 현대 아울렛점/주니어,시니어/신입가능/주5일근무",상시모집,00명(인원미정),NaN,"대전 유성구 테크노중앙로 123 (용산동, 현대프리미엄아울렛대전점) 명품관 마이클코어스",성별무관,학력무관,연령무관,"유사업무 경험 우대, 인근 거주 우대"
2,[PXG] | 갤러리아타임월드에서 시니어를 모집합니다,상시모집,1명,NaN,"대전 서구 대덕대로 211 (둔산동, 갤러리아타임월드백화점) 7층PXG",성별무관,학력무관,28세 ~ 45세,NaN
3,[신세계 대전점 디스커버리 키즈] | [신세계백화점 디스커버리 키즈] 매장관리 및 ...,2023-10-30마감 16일전,0명(인원미정),NaN,대전 유성구 엑스포로 1 (도룡동)대전 유성구 엑스포로 1,성별무관,학력무관,연령무관,유사업무 경험 우대
4,[호야 & 피플] | [SW19] 갤러리아 타임월드점 시니어 채용합니다.,상시모집,1명,NaN,전국,성별무관,학력무관,연령무관,NaN
5,[㈜룸페커] | (주)룸페커 대전노은 직영점 매장 시니어 정직원 구인,2023-10-31마감 17일전,1명,"매장판매, 매장관리",대전 유성구 은구비남로33번길 51 (지족동) 1층,성별무관,고등학교 졸업 이상,연령무관,"유사업무 경험 우대, 인근 거주 우대"
6,[㈜ 에프에이모스트] | 스말트 롯데백화점 대전점 시니어 / 파트타이머 구인 (남...,상시모집,3명,"매니저, 시니어, 주3일 파트타이머","대전 서구 계룡로 598 (괴정동, 롯데백화점) 롯데백화점 대전점 1층 SMALT",성별무관,학력무관,연령무관,"영어 가능, 엑셀 가능, 유사업무 경험 우대"
7,[㈜휴머니스트] | [정규직/경력별협의] 여성의류 룸페커 대전 노은점 시니어 채용,2023-10-28마감 14일전,0명(인원미정),NaN,대전 유성구 은구비남로33번길 51 (지족동) 룸페커 매장,성별무관,학력무관,연령무관,"영어 가능, 일본어 가능, 중국어 가능, 컴퓨터 활용 가능, 업무 관련 자격증 소지..."
8,[㈜휴머니스트] | [TAGHEUER]명품 시계 태그호이어 신세계 대전점 시니어/주...,2023-10-28마감 14일전,00명(인원미정),NaN,"대전 유성구 대덕대로 480 (도룡동, 신세계 대전점) 태그호이어 매장",성별무관,학력무관,연령무관,"영어 가능, 일본어 가능, 중국어 가능, 컴퓨터 활용 가능, 업무 관련 자격증 소지..."
9,[㈜휴머니스트] | [MICHAEL KORS] 명품 마이클코어스코리아 대전아울렛 시...,2023-10-17마감 3일전,00명(인원미정),NaN,"대전 유성구 테크노중앙로 123 (용산동, 현대프리미엄아울렛대전점) 마이클코어스 매장",성별무관,학력무관,연령무관,"영어 가능, 일본어 가능, 중국어 가능, 컴퓨터 활용 가능, 업무 관련 자격증 소지..."


In [32]:
driver.close()
driver.quit()